In [ ]:
# imports
import shutil
from pathlib import Path

from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

In [ ]:
# set network path and get list of directories to process
data_directory = Path('/Users/dlisla/Pictures')
input_directory = data_directory.joinpath('1.toOCR')
output_directory = data_directory.joinpath('2.OCRed')

directories_to_convert_to_pdf_paths_list = sorted([x for x in input_directory.iterdir() if x.is_dir()])

print(f'{len(directories_to_convert_to_pdf_paths_list)} directories to process')

In [ ]:
# functions

def get_tif_paths_list(directory_path, scantailor=False):
    
    # if directory_path not a Path-like object then make it one
    if not isinstance(directory_path, Path):
        directory_path = Path(directory_path)
        
    # add 'out' path to directory if processed with ScanTailor
    if scantailor:
        directory_path = Path(directory_path).joinpath('out')
    
    # get paths to *.tif to process
    tif_paths_list = sorted(directory_path.glob('*.tif'))
    
    if len(tif_paths_list) == 0:
        tif_paths_list = sorted(directory_path.glob('*.tiff'))
    
    # only keep *.tif that DO NOT start with '.' to exclude macOS index files
    tif_paths_list = [x for x in tif_paths_list if not str(x.name).startswith('.')]
    
    return tif_paths_list


def create_temporary_pdf_directory(directory_path):
    
    # if directory_path not a Path-like object then make it one
    if not isinstance(directory_path, Path):
        directory_path = Path(directory_path)
        
    temporary_pdf_directory_path = Path.home().joinpath(f'Desktop/local_temporary_pdfs/_temporary_pdfs_{directory_path.name}')
    
    try:  # creating the temporary directory
        temporary_pdf_directory_path.mkdir(parents=True)
    except FileExistsError:  # delete the directory & all contents then create it
        shutil.rmtree(temporary_pdf_directory_path)
        temporary_pdf_directory_path.mkdir()
        
    return temporary_pdf_directory_path


def batch_ocr_tifs_into_pdfs(tif_paths_list, temporary_pdf_directory_path):
    
    number_of_tifs = len(tif_paths_list)
    number_of_pdfs = 0
    
    # progress bar
    progress_label = Label('OCR *.tif')
    progress_bar = IntProgress(min=0, max=number_of_tifs)
    progress_widget = VBox([progress_label, progress_bar])
    display(progress_widget)
    
    for tif_path in tif_paths_list:
        
        label = f'OCRing {tif_path.name}'
        progress_label.value = label

        temporary_pdf_output_path = temporary_pdf_directory_path.joinpath(tif_path.stem)

        # ocr *.tif with tesseract
        # NOTE: output SHOULD NOT have extension on end!
        !tesseract "{str(tif_path)}" "{str(temporary_pdf_output_path)}" pdf 2>/dev/null

        # Add the suffix .pdf that tesseract didn't want
        temporary_pdf_output_path = temporary_pdf_directory_path.joinpath(f'{tif_path.stem}.pdf') 

        if temporary_pdf_output_path.is_file():
            number_of_pdfs += 1
            progress_bar.value = number_of_pdfs
        
    if number_of_pdfs != number_of_tifs:
        print(f'ERROR - ERROR - ERROR')
        print(f'# of *.tif: {number_of_tifs}')
        print(f'# of PDFs: {number_of_pdfs}')
    else:
        print(f'{number_of_pdfs} PDFs created and saved in {temporary_pdf_directory_path}')
    
    
def concatenate_pdfs(temporary_pdf_directory_path):

    # strip '_temporary_pdfs_' from beginning of directory path to get combined PDF name
    combined_pdf_name = f"{str(temporary_pdf_directory_path).split('_temporary_pdfs_')[1]}.pdf"

    combined_pdf_output_path = temporary_pdf_directory_path.joinpath(combined_pdf_name)

    # concatenate with pdftk
    !pdftk "{str(temporary_pdf_directory_path)}"/*.pdf cat output "{str(combined_pdf_output_path)}"

    if combined_pdf_output_path.is_file() and combined_pdf_output_path.stat().st_size > 0:
        print(f'{combined_pdf_output_path} created with size {round((combined_pdf_output_path.stat().st_size/1024/1024/1024), 2)} GB')

    return combined_pdf_output_path

In [4]:
directories_to_convert_to_pdf_paths_list

[PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1978-10-20_EX1-37'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-03-05_EX1-49'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-05-04_EX1-13'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-06-20_EX1-13'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-06-21_EX1-59'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-07-11_EX1-2'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-09-10_EX1-4'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-09-28_EX1-43'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-10-29_EX1-2'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1979-12-12_EX1-12'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1980-02-08_EX1-38'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1980-04-25_EX1-16'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1980-06-18_EX1-10'),
 PosixPath('/Users/dlisla/Pictures/1.toOCR/R_ BOT_1980-06-19_EX1-67'),
 PosixPat

In [5]:
for dir_path in reversed(directories_to_convert_to_pdf_paths_list):
    
    # get list of *.tif to process
    tif_paths_list = get_tif_paths_list(dir_path)
    
    number_of_tifs = len(tif_paths_list)
    print(f'{number_of_tifs} *.tif to process in {dir_path}')
    
    # create temp directory
    temp_pdf_dir_path = create_temporary_pdf_directory(dir_path)
    
    # OCR tifs into PDFs with progress bar
    batch_ocr_tifs_into_pdfs(tif_paths_list, temp_pdf_dir_path)
    
    # concatenate PDFs into single file
    combined_pdf_path = concatenate_pdfs(dir_path)
    
    # copy combined PDF to copy directory
    output_pdf_path = output_directory.joinpath(combined_pdf_path.name)
    shutil.copy(combined_pdf_path, output_pdf_path)
    
    if output_pdf_path.is_file():  # move processed dir_path into output_directory
        renamed_dir_path = output_directory.joinpath(dir_path.name)
        dir_path.rename(renamed_dir_path)

    print('')
    print('*****')    
    print(f'{dir_path.name} finished')
    print('*****')
    print('')

604 *.tif to process in /Users/dlisla/Pictures/1.toOCR/R_ BOT_1982-10-15_EX1-38


604 PDFs created and saved in /Users/dlisla/Desktop/local_temporary_pdfs/_temporary_pdfs_R_ BOT_1982-10-15_EX1-38


IndexError: list index out of range

In [ ]:
dir_path = Path('/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1977-06-16_EX1-76')
dir_path

In [ ]:
combined_pdf_path = concatenate_pdfs(dir_path)

In [ ]:
# open output_directory_path for visual Quality Assurance
!open {str(output_directory)}

In [ ]:
# delete all '_temporary_pdfs_*' directories on the desktop
all_temp_dir_paths = sorted(Path('/Users/dlisla/Desktop/').glob('_temporary_pdfs_*'))
number_of_dir_paths = len(all_temp_dir_paths)
for temp_dir_path in all_temp_dir_paths:
    shutil.rmtree(temp_dir_path)

all_temp_dir_paths = sorted(Path('/Users/dlisla/Desktop/').glob('_temporary_pdfs_*'))
number_of_deleted_dir_paths = number_of_dir_paths - len(all_temp_dir_paths)

print(f'{number_of_deleted_dir_paths} _temporary_pdf_* directories deleted')

In [ ]:
temp_pdf_dir_path

In [ ]:
pdf_paths_list = sorted(temp_pdf_dir_path.glob('*.pdf'))
len(pdf_paths_list)

In [ ]:
max_pdfs_to_combine = 999
count = 0
for index, pdf_path in enumerate(pdf_paths_list, start = 1):
    sub_dir_path = temp_pdf_dir_path.joinpath(f'pdfs_to_combine_{str(count).zfill(2)}')
    if not sub_dir_path.is_dir():
        sub_dir_path.mkdir()
    
    # move pdf to that directory
    pdf_path.rename(sub_dir_path.joinpath(pdf_path.name))
    
    
    if index % max_pdfs_to_combine == 0:
        count += 1

In [ ]:
directories_to_convert_to_pdf_paths_list = sorted([x for x in temp_pdf_dir_path.iterdir() if x.is_dir()])
len(directories_to_convert_to_pdf_paths_list)